In [13]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools

In [14]:
from sklearn.model_selection import StratifiedKFold
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:3]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [22]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=90.0):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

#geometric mean
def score(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,len(y_test)):
        if(y_test[i]==1):
            test_min=test_min+1
        else:
            test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,len(y_predict)):
        if(y_predict[i]==1 and y_predict[i]==y_test[i]):
            pred_min=pred_min+1
        elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
            pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)   

In [63]:
from cvxopt import matrix
class HYP_SVM(object):

    #geometric mean
    def score(y_predict,y_test):
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        print("y_test min",test_min)       
        print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        print("y_pred min",pred_min)       
        print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm)    
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        self.sigma = sigma
        if self.C is not None: self.C = float(self.C)
            
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma, "kernel": self.kernel}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train,X_test, y):
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features= X_test.shape
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        X_test=np.asarray(X_test)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:115]*r_min
        print(self.m.shape)
        self.m=np.append(self.m,func[115:5473]*r_max)
        print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 
        #nt_samples, nt_features = X_test.shape
        # Gram matrix

        print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None        
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
                print(y_predict[i]+self.b)
                #print(self.b)
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [24]:
if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        C_vals = [1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4]
        sigma = [9e-2, 9e-1, 9e+1, 9e+2, 9e+3, 9e+4]
        combs = list(itertools.product(C_vals, sigma))
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        skf2 = StratifiedKFold(n_splits=len(combs), shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            i=0
            for train_index, test_index in skf2.split(X_train, y_train):
                print("\n\n Using C=", combs[i][0], "and sigma=", combs[i][1])
                clf = HYP_SVM(C=combs[i][0], sigma=combs[i][1])
                #clf = HYP_SVM(C=combs[i][0], kernel=gaussian_kernel, sigma=combs[i][1])
                i+=1
                #typ = 2            
                
                #print(len(X_train), len(X_test), len(y_train))
                clf.m_func(X_train,X_test,y_train)
                
                clf.fit(X_train, y_train)
                y_predict = clf.predict(X_test)
                print(y_test)
                score(y_predict,y_test)
                correct = np.sum(y_predict == y_test)
                mse = mean_squared_error(y_test, y_predict)
                print("%d out of %d predictions correct" % (correct, len(y_predict)))
                print("Accuracy",correct/len(y_predict))
                print("Errore quadratico medio: ", mse)
            
    hyp_svm()    



 Using C= 0.01 and sigma= 0.09
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -1.8532e+01 -8.3103e+00  5e+02  2e+01  3e-16
 1: -7.2111e+00 -3.0922e+00  4e+01  2e+00  4e-16
 2: -8.0984e-01 -2.2334e+00  2e+00  2e-02  1e-15
 3: -7.8553e-01 -8.5733e-01  7e-02  2e-04  4e-16
 4: -7.9153e-01 -7.9821e-01  7e-03  2e-05  4e-16
 5: -7.9264e-01 -7.9355e-01  9e-04  1e-06  3e-16
 6: -7.9283e-01 -7.9296e-01  1e-04  6e-08  3e-16
 7: -7.9285e-01 -7.9287e-01  1e-05  2e-09  3e-16
 8: -7.9286e-01 -7.9286e-01  8e-07  5e-11  3e-16
 9: -7.9286e-01 -7.9286e-01  4e-08  7e-13  4e-16
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -1.8532e+01 -8.3103e+00  5e+02  2e+01  3e-16
 1: -7.2111e+00 -3.0922e+00  4e+01  2e+00  4e-16
 2: -8.0984e-01 -2.2334e+00  2e+00  2e-02  1e-15
 3: -7.8553e-01 -8.5733e-01  7e-02  2e-04  4e-16
 4: -7.9153e-01 -7.9821e-01  7e-03  2e-05  4e-16
 5: -7.9264e-01 -7.9355e-01  9e-04  1e-06  3e-16
 6: -7.9283e-01 -7.

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3867e+01 -2.5347e+00  6e+02  3e+01  3e-15
 1: -1.3258e+00 -2.5227e+00  8e+00  3e-01  4e-15
 2: -7.8964e-01 -1.7596e+00  1e+00  4e-17  8e-16
 3: -7.9977e-01 -8.1102e-01  1e-02  7e-17  7e-16
 4: -8.0000e-01 -8.0011e-01  1e-04  5e-17  6e-16
 5: -8.0000e-01 -8.0000e-01  1e-06  3e-16  5e-16
 6: -8.0000e-01 -8.0000e-01  2e-08  2e-16  7e-16
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3867e+01 -2.5347e+00  6e+02  3e+01  3e-15
 1: -1.3258e+00 -2.5227e+00  8e+00  3e-01  4e-15
 2: -7.8964e-01 -1.7596e+00  1e+00  4e-17  8e-16
 3: -7.9977e-01 -8.1102e-01  1e-02  7e-17  7e-16
 4: -8.0000e-01 -8.0011e-01  1e-04  5e-17  6e-16
 5: -8.0000e-01 -8.0000e-01  1e-06  3e-16  5e-16
 6: -8.0000e-01 -8.0000e-01  2e-08  2e-16  7e-16
Optimal solution found.
optimal
-0.3333333323987813
-0.33333332634835877
-0.3333333280744276
-0.33333332800890897
-0.33333332982054703
-0.333333327093

-0.32200340089645224
-0.3222330792323991
-0.3220638802068202
-0.3221340078132655
-0.32222213254941284
-0.3229218132333312
-0.32254114457626765
-0.32260161297862056
-0.3223032011317773
-0.32208188284616085
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
y_test min 10
y_test max 20
y_pred min 0
y_pred max 20
0.0 1.0
GM 0.0
20 out of 30 predictions correct
Accuracy 0.6666666666666666
Errore quadratico medio:  1.3333333333333333


 Using C= 0.1 and sigma= 900.0
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.8664e+01 -2.5467e+01  6e+02  2e+01  4e-15
 1: -8.3920e+00 -2.4302e+01  2e+01  2e-01  4e-15
 2: -7.9459e+00 -8.8054e+00  9e-01  9e-16  1e-15
 3: -7.9994e+00 -8.0082e+00  9e-03  1e-15  1e-15
 4: -8.0000e+00 -8.0001e+00  1e-04  6e-16  1e-15
 5: -8.0000e+00 -8.0000e+00  2e-05  3e-15  1e-15
 6: -8.0000e+00 -8.0000e+00  4e-06  3e-15  2e-15
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
    

KeyboardInterrupt: 

When assigning fuzzy memberships for training examples
in the FSVM-CIL method, in order to reflect the class imbal-
ance, we assign r + = 1, and r − = r, where r is the minority-
to-majority class ratio. This was following the findings reported
in [5], where the optimal results for the DEC method could
be obtained when C − /C + equals to the minority-to-majority
class ratio. According to this assignment of values, a positive-
class example can take a membership value in the [0, 1] interval,
while a negative-class example can take a membership value in
the [0, r] interval, where r < 1.

[...]

selected the
following ranges for parameters: log 2 C = {1, 2, . . . , 15}, and
log 2 γ = {−15, −14, . . . , −1}. First, we conducted a coarse
grid-parameter-search to find the optimal values for log 2 C and
log 2 γ over the aforementioned ranges of values. We evalu-
ated the performance of a model trained on each parameter
pair (log 2 C, log 2 γ) by using five-fold cross-validation results
on the training dataset. After finding the optimal values for
the parameters, say ( C̄,γ̄), again, a narrow grid-parameter-
search was conducted over the ranges log 2 C = {c̄ − 0.75, c̄ −
0.5, . . . , c̄ + 0.75}, and log 2 γ = {γ̄ − 0.75, γ̄ − 0.5, . . . , γ̄ +
0.75}. Finally, after finding the optimal values for (log 2 C,
log 2 γ), a new SVM model was trained by using the complete
training dataset on these parameter values

In [ ]:
clf_svm = svm.SVC(kernel='rbf', gamma=0.001, C=100)
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test) 
acc_svm = accuracy_score(y_test, y_pred_svm)
print ("Overall RBF KERNEL SVM accuracy: ",acc_svm)

In [ ]:
if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        C_vals = [1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4]
        sigma = [9e-2, 9e-1, 9e+1, 9e+2, 9e+3, 9e+4]
        
        parameters = {'C': C_vals, 'sigma': sigma }
        clf = GridSearchCV(model, parameters, cv = 5)
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        skf2 = StratifiedKFold(n_splits=len(combs), shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            i=0
            for train_index, test_index in skf2.split(X_train, y_train):
                print("\n\n Using C=", combs[i][0], "and sigma=", combs[i][1])
                clf = HYP_SVM(C=combs[i][0], kernel=gaussian_kernel, sigma=combs[i][1])
                i+=1
                typ = 2            
    
                clf.m_func(X_train,X_test,y_train)
                clf.fit(X_train,X_test, y_train)
                y_predict = clf.predict(X_test)
                print(y_test)
                gm(y_predict,y_test)
                correct = np.sum(y_predict == y_test)
                mse = mean_squared_error(y_test, y_predict)
                print("%d out of %d predictions correct" % (correct, len(y_predict)))
                print("Accuracy",correct/len(y_predict))
                print("Errore quadratico medio: ", mse)
            
    hyp_svm() 

In [66]:
C_vals = [1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9e+1, 9e+2, 9e+3, 9e+4]
kernels = ["linear_kernel", "polynomial_kernel", "gaussian_kernel"]
parameters = {'C': C_vals, 'sigma': sigma, 'kernel' : kernels }

model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)

xTrain, xTest, yTrain, yTest = train_test_split(X,y,  test_size=30)
model.m_func(xTrain,xTest,yTrain)

clf = GridSearchCV(model, parameters, cv = 5, return_train_score=True)

clf.fit(xTrain, yTrain)
#clf.score(xTest, yTest)
#clf.best_params_

#best_model = FuzzyMMC(sensitivity=0.5, exp_bound=0.60, animate=False)
#best_model.fit(xTrain, yTrain)

(120, 120)
     pcost       dcost       gap    pres   dres
 0:  2.1631e+04 -2.2629e+05  4e+05  1e-01  3e-14
 1:  1.2065e+04 -2.6367e+04  5e+04  1e-02  6e-14
 2:  2.7925e+03 -4.7250e+03  8e+03  1e-03  3e-14
 3:  3.9162e+02 -6.1240e+02  1e+03  1e-14  2e-14
 4:  3.4389e+01 -1.0118e+02  1e+02  1e-15  8e-15
 5: -3.9411e+00 -5.2880e+01  5e+01  2e-15  3e-15
 6: -2.3726e+01 -4.6685e+01  2e+01  4e-16  4e-15
 7: -2.9957e+01 -3.4971e+01  5e+00  4e-15  5e-15
 8: -3.1334e+01 -3.2913e+01  2e+00  4e-15  3e-15
 9: -3.1961e+01 -3.2104e+01  1e-01  1e-14  4e-15
10: -3.2023e+01 -3.2025e+01  1e-03  1e-14  4e-15
11: -3.2024e+01 -3.2024e+01  1e-05  6e-15  4e-15
Optimal solution found.
optimal
(115,)
(120,)


AttributeError: 'HYP_SVM' object has no attribute 'K'

In [ ]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:
if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)    
            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            mse = mean_squared_error(y_test, y_predict)
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print("Accuracy",correct/len(y_predict))
            print("Errore quadratico medio: ", mse)
    normal_svm()   